## #다음 testfile1.txt, testfile2.txt, testfile3.txt에 대해서 질문에 답하시오. 
1. testfile1.txt 에 대해서 각 사람별 평균을 구하시오.
2. testfile2.txt, testfile3.txt 에 공통적으로 나타나는 단어에 대해 빈도수를 내림차순으로 출력하시오. 

In [3]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder.master("local").appName("lab5").getOrCreate()

#### 1번 문제

In [2]:
rdd = spark_session.sparkContext.textFile("./data/testfile1.txt")
rdd.take(2)

['seokil, 50', 'seokil, 60']

In [11]:
rdd1 = rdd.map(lambda x: x.split(',')).map(lambda x: (x[0], (int(x[1]), 1)))
rdd1.take(2)

[('seokil', (50, 1)), ('seokil', (60, 1))]

In [12]:
rdd2 = rdd1.reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))
rdd2.take(2)

[('seokil', (110, 2)), ('gildong', (140, 2))]

In [15]:
rdd3 = rdd2.mapValues(lambda x: x[0]/x[1])
rdd3.take(2)

[('seokil', 55.0), ('gildong', 70.0)]

#### 2번 문제

In [4]:
t2rdd = spark_session.sparkContext.textFile("./data/testfile2.txt")
t3rdd = spark_session.sparkContext.textFile("./data/testfile3.txt")
print(t2rdd.take(2))
print(t3rdd.take(2))

["pairRDD = sc.parallelize( [('panda', 0), ('pink', 3), ('pirate', 3), ('panda', 1), ('pink', 4)] )", 'result1 = pairRDD.groupByKey()']
['sc = SparkContext(conf=conf)', '']


In [9]:
import sys, re

t2rdd_wordcnt = t2rdd.filter(lambda line: len(line) > 0) \
			   .flatMap(lambda line: re.split('\W+', line)) \
			   .filter(lambda word: len(word) > 1) \
			   .map(lambda word:(word.lower(),1)) \
			   .reduceByKey(lambda v1, v2: v1 + v2) \
			   .persist()
t2rdd_wordcnt.take(6)

[('pairrdd', 2),
 ('sc', 1),
 ('parallelize', 1),
 ('panda', 2),
 ('pink', 2),
 ('pirate', 1)]

In [6]:
t3rdd_wordcnt = t3rdd.filter(lambda line: len(line) > 0) \
			   .flatMap(lambda line: re.split('\W+', line)) \
			   .filter(lambda word: len(word) > 1) \
			   .map(lambda word:(word.lower(),1)) 
			   .reduceByKey(lambda v1, v2: v1 + v2) \
			   .persist()
t3rdd_wordcnt.take(5)

[('sc', 1), ('sparkcontext', 1), ('conf', 1), ('conf', 1), ('pairrdd1', 1)]

In [33]:
result = t2rdd_wordcnt.join(t3rdd_wordcnt).mapValues(lambda x: x[0]+x[1]) \
            .sortBy(lambda x: x[1], ascending=False)
result.take(5)

[('result1', 5), ('sc', 4), ('panda', 4), ('pink', 4), ('parallelize', 3)]